In [ ]:
from inference import get_model
#from google.colab import userdata

ROBOFLOW_API_KEY = "ITfUpuY5QO9WTBpcEXTh"  # Replace with your actual Roboflow API key

PLAYER_DETECTION_MODEL_ID = "football-players-detection-3zvbc/12"
PLAYER_DETECTION_MODEL = get_model(model_id=PLAYER_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

Simple annotation with Boxes

In [ ]:
SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/videos/goalMatch.mp4"


In [ ]:
import supervision as sv

box_annotator = sv.BoxAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000')
)


In [ ]:

frame_number = 0  # Change to the desired frame number
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

for i, frame in enumerate(frame_generator):
    if i == frame_number:
        # Run inference on the specific frame
        result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
        detections = sv.Detections.from_inference(result)

        # Annotate the frame
        annotated_frame = frame.copy()
        annotated_frame = box_annotator.annotate(
            scene=annotated_frame,
            detections=detections
        )
        
        labels = [
          f"{class_name} {confidence:.2f}"
          for class_name, confidence
          in zip(detections["class_name"] , detections.confidence)
         ]
        annotated_frame = label_annotator.annotate(annotated_frame , detections,labels =labels)##annotate the frame with the labels

         # Show the annotated frame
        sv.plot_image(annotated_frame)
        break  # Stop after processing the specified frame


GENERATING VIDEO WITH ANNOTATIONS

In [ ]:
from tqdm import tqdm
import supervision as sv

SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/121364_0.mp4"
TARGET_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/121364_0_result.mp4"

box_annotator = sv.BoxAnnotator(##Box annotator options like coloring and thickness
    color = sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
    thickness = 2
)


In [ ]:
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)##I will take the info from the source video such as resolution and frame rate to pass into the result video 
video_sink = sv.VideoSink(TARGET_VIDEO_PATH, video_info=video_info)##The result video
frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)


In [ ]:



with video_sink:
     for frame in tqdm(frame_generator , total = video_info.total_frames):



      label_annotator = sv.LabelAnnotator(##labeling the box and adding text
          color = sv.ColorPalette.from_hex(['#FF8C00', '#00BFFF', '#FF1493', '#FFD700']),
          text_color = sv.Color.from_hex('#000000')
      )

      frame = next(frame_generator)

      result = PLAYER_DETECTION_MODEL.infer(frame,confidence = 0.3)[0]##This give me a row data of the detection
      detections = sv.Detections.from_inference(result)##Converting the row data into standarized format


      labels = [
          f"{class_name} {confidence:.2f}"
          for class_name, confidence
          in zip(detections["class_name"] , detections.confidence)
      ]

      annotated_frame = frame.copy()##Copying the frame to anotate it with boxes
      annotated_frame = box_annotator.annotate(annotated_frame , detections)##annotate the frame with the detections
      annotated_frame = label_annotator.annotate(annotated_frame , detections,labels =labels)##annotate the frame with the labels
      video_sink.write_frame(annotated_frame)##writing the annotated frame and looping over all video frame 

In [ ]:
import supervision as sv
from TeamClassifier import TeamClassifier

SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/121364_0.mp4"

BALL_ID = 0
PLAYER_ID = 2

crops = extract_crops(SOURCE_VIDEO_PATH)

team_classifier = TeamClassifier(device=DEVICE)
team_classifier.fit(crops)

ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']),
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)

tracker = sv.ByteTrack()
tracker.reset()

#Classify the teams

In [ ]:

frame_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(frame_generator)

result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
detections = sv.Detections.from_inference(result)

ball_detections = detections[detections.class_id == BALL_ID]
ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)

all_detections = detections[detections.class_id != BALL_ID]
all_detections = all_detections.with_nms(threshold=0.5, class_agnostic=True)
all_detections = tracker.update_with_detections(detections=all_detections)

players_detections = all_detections[all_detections.class_id == PLAYER_ID]
players_crops = [sv.crop_image(frame,xyxy) for xyxy in players_detections.xyxy]
players_detections.class_id = team_classifier.predict(players_crops)

labels = [
    f"#{tracker_id}"
    for tracker_id
    in players_detections.tracker_id
]

annotated_frame = frame.copy()
annotated_frame = ellipse_annotator.annotate( scene=annotated_frame, detections = players_detections)
annotated_frame = label_annotator.annotate(scene=annotated_frame,detections = players_detections, labels=labels)
annotated_frame = triangle_annotator.annotate( scene=annotated_frame, detections = ball_detections)

sv.plot_image(annotated_frame)

In [ ]:
from tqdm import tqdm 
SOURCE_VIDEO_PATH = "C:/Users/user/Desktop/FootballAI/121364_0.mp4"

STRIDE = 30 
PLAYER_ID =2
def extract_crops(source_video_path : str):
        frame_generator = sv.get_video_frames_generator(source_video_path, stride=STRIDE)##Getting the frames with a stride of 30

        crops = []

        for frame in tqdm(frame_generator , desc="Extracting crops"):
            result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.5)[0]
            detections = sv.Detections.from_inference(result)
            detections = detections.with_nms(threshold=0.5 , class_agnostic=True)
            detections = detections[detections.class_id == PLAYER_ID]

            crops += [
                sv.crop_image(frame,xyxy)
                for xyxy
                in detections.xyxy
            ]
        return crops